In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))


import pandas as pd
from preprocess import  preprocess_metadata
from semantic_search import semantic_search ,enhanced_search, generate_embeddings

## Data Exploration

### Queries

In [4]:
queries = pd.read_csv("../data/queries.csv")

In [3]:
queries

,search_term_pt
0,Batatas fritas de rua carregadas
1,Pizza de massa fina assada em forno a lenha
2,Sopa de macarrão feita à mão
3,Almoço estilo havaiano
4,Sanduíche de café da manhã com abacate
...,...
95,Arroz temperado do norte da África
96,Prato para almoço de escritório
97,Caixa de proteína rápida e saudável
98,Carne assada caseira com acompanhamentos


In [33]:
queries.describe()

,search_term_pt
count,100
unique,100
top,Batatas fritas de rua carregadas
freq,1


### Items

In [5]:
items = pd.read_csv("../data/5k_items_curated.csv")

In [11]:
items

,_id,itemId,itemMetadata,itemProfile,merchantId
0,6813b2b5b7476b2ec454b7ff,442731c2-144b-4c5c-b81e-e0cf9dfa9630,"{""category_name"":""Alimentos Básicos"",""descript...","{""metrics"":{""avgBasketPrice"":159.89,""coPurchas...",d6cf7b73-c9bb-4c8a-abd6-c1a832bdd461
1,6813b2b5b7476b2ec454b800,4a6d4a8f-c1bf-4457-a448-4a3b5e1eb20c,"{""category_name"":""Doces e Sobremesas"",""descrip...","{""metrics"":{""avgBasketPrice"":182.44,""coPurchas...",d6cf7b73-c9bb-4c8a-abd6-c1a832bdd461
2,6813b2b5b7476b2ec454b801,5380e263-c202-45cf-8eae-81e54573f122,"{""category_name"":""Higiene e Cuidados Pessoais""...","{""metrics"":{""avgBasketPrice"":190.63,""coPurchas...",d6cf7b73-c9bb-4c8a-abd6-c1a832bdd461
3,6813b2b5b7476b2ec454b802,595ce305-ddf5-49fc-9a8c-b50a9ccf6267,"{""category_name"":""Higiene e Cuidados Pessoais""...","{""metrics"":{""avgBasketPrice"":113.56,""coPurchas...",d6cf7b73-c9bb-4c8a-abd6-c1a832bdd461
4,6813b2b5b7476b2ec454b803,5e2db3a9-1a2c-4462-8aa1-1ed045122e12,"{""category_name"":""Pet Shop"",""description"":""• R...","{""metrics"":{""avgBasketPrice"":223.58,""coPurchas...",d6cf7b73-c9bb-4c8a-abd6-c1a832bdd461
...,...,...,...,...,...
4995,6813b39eb7476b2ec454cb82,d4fe2d30-a824-4071-8bb7-9e70ab984db6,"{""category_name"":""Croissant & Brezel"",""descrip...","{""metrics"":{""avgBasketPrice"":78.99,""coPurchase...",bf0d36fe-8972-4b66-8247-38aba1254ae1
4996,6813b39eb7476b2ec454cb83,3aeb7744-c99d-43dc-883f-4eec44d7d85b,"{""category_name"":""Lanches Saudáveis"",""descript...","{""metrics"":{""avgBasketPrice"":80.68,""coPurchase...",bf2834a7-754f-48cc-944f-cb14e65d580c
4997,6813b39eb7476b2ec454cb84,cf9963da-6971-4acb-a511-4663a4a500a7,"{""category_name"":""Bebidas"",""description"":""Garr...","{""metrics"":{""avgBasketPrice"":28.2,""coPurchaseI...",bf3d038c-eb28-419e-81c4-a8475bbea9b7
4998,6813b39eb7476b2ec454cb85,137495b9-7098-4fe4-8992-db2ab1c2e2ad,"{""category_name"":""Sucos Naturais e Especiais"",...","{""metrics"":{""avgBasketPrice"":80.16,""coPurchase...",bf78c130-3ae9-4d46-b600-ae09849099ab


In [12]:
items.describe()

,_id,itemId,itemMetadata,itemProfile,merchantId
count,5000,5000,5000,5000,5000
unique,5000,4997,4968,4997,1806
top,6813b2b5b7476b2ec454b7ff,fd7d8fe6-1b23-4bf9-b23b-0d978f904492,"{""category_name"":""Limpeza"",""description"":"""",""i...","{""metrics"":{""avgBasketPrice"":140.34,""coPurchas...",f91fa0ce-57d6-4a3d-95be-79ef1ab71f23
freq,1,4,4,4,73


In [13]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           5000 non-null   object
 1   itemId        5000 non-null   object
 2   itemMetadata  5000 non-null   object
 3   itemProfile   5000 non-null   object
 4   merchantId    5000 non-null   object
dtypes: object(5)
memory usage: 195.4+ KB


In [15]:
items['itemMetadata'][0]

'{"category_name":"Alimentos Básicos","description":"Pacote 500g","images":["820af392-002c-47b1-bfae-d7ef31743c7f/202210182253_3h93mu9eg9y.jpg","820af392-002c-47b1-bfae-d7ef31743c7f/202210182253_06c38y2upker.jpg"],"lacFree":false,"name":"Macarrão Pena com Ovos Adria 500g","organic":false,"price":3.56,"tags":[{"key":"PORTION_SIZE","value":["NOT_APPLICABLE"]}],"taxonomy":{"l0":"MERCEARIA","l1":"MASSAS_SECAS","l2":"MASSA_MACARRAO"},"vegan":false}'

In [16]:
items['itemProfile'][0]

'{"metrics":{"avgBasketPrice":159.89,"coPurchaseItems":[{"gross_price":4.79,"item_description":"","item_id":"815bf4a6-5478-4663-a4a9-e45b165f9928","item_name":"Leite Uht Integral Italac 1l","merchant_id":"d6cf7b73-c9bb-4c8a-abd6-c1a832bdd461"},{"gross_price":15.0,"item_description":"Embalagem 1un","item_id":"95ad7643-9acb-48f6-a75c-f4bcffbf3784","item_name":"Ovo Grande Vermelho C12 Carrefour","merchant_id":"d6cf7b73-c9bb-4c8a-abd6-c1a832bdd461"},{"gross_price":6.05,"item_description":"Embalagem 170g","item_id":"93e2ae19-990a-486f-b5b3-4c693ac19c1e","item_name":"Atum Ralado Ao Natural Carrefour 170g","merchant_id":"d6cf7b73-c9bb-4c8a-abd6-c1a832bdd461"},{"gross_price":5.24,"item_description":"","item_id":"b7259e0f-34dd-445e-b550-ab4049df85b3","item_name":"Leite Integral Uht Piracanjuba 1l","merchant_id":"d6cf7b73-c9bb-4c8a-abd6-c1a832bdd461"},{"gross_price":2.27,"item_description":"Embalagem 300g","item_id":"2e975f77-5fcc-43eb-9bf2-af1506b63d67","item_name":"Molho de Tomate Sabor Manjer

## Feature Engineering

In [6]:
import json
items['jointText'] = items.apply(lambda row: preprocess_metadata(row['itemMetadata'], row['itemProfile'],type='Basic'), axis=1)
items['jointTextNatural'] = items.apply(lambda row: preprocess_metadata(row['itemMetadata'], row['itemProfile'],type='Natural'), axis=1)

In [7]:
items['organic'] = items.apply(lambda row: json.loads(row['itemMetadata'])['organic'], axis=1)
items['lacFree'] = items.apply(lambda row: json.loads(row['itemMetadata'])['lacFree'], axis=1)
items['vegan'] = items.apply(lambda row: json.loads(row['itemMetadata'])['vegan'], axis=1)
items['avgBasketPrice'] = items.apply(lambda row: json.loads(row['itemProfile'])["metrics"]['avgBasketPrice'], axis=1)
items['total_orders'] = items.apply(lambda row: json.loads(row['itemProfile'])["metrics"]['total_orders'], axis=1)
items['category_name'] = items.apply(lambda row: json.loads(row['itemMetadata'])['category_name'], axis=1)

In [8]:
reorderRate=[]
for i, row in items.iterrows():
    try:
        reorderRate.append(json.loads(row['itemProfile'])["metrics"]["reorderRate"])
    except:
        reorderRate.append(0.0)
items['reorderRate']=reorderRate

In [9]:
embeddings_items =generate_embeddings(items['jointText'])
embeddings_items_natural =generate_embeddings(items['jointTextNatural'])
query_gallery_embedding=generate_embeddings(queries['search_term_pt'])

In [10]:
items['embeddings_jointText']=embeddings_items
items['embeddings_jointTextNatural']=embeddings_items_natural

In [13]:
items.to_csv("../data/5k_items_processed.csv", index=False)

In [14]:
items['organic'].value_counts()

organic
False    4972
True       28
Name: count, dtype: int64

In [15]:
items['lacFree'].value_counts()

lacFree
False    4961
True       39
Name: count, dtype: int64

In [16]:
items['vegan'].value_counts()

vegan
False    4977
True       23
Name: count, dtype: int64

In [17]:
items['category_name'].value_counts()

category_name
Medicamentos                  483
Bebidas                       346
Higiene e Beleza              183
Limpeza                       181
Doces e Sobremesas            166
                             ... 
Potes de açaí                   1
Boquinha de Anjo                1
Master Hot Dogs Saborosos.      1
Clássicos na medida             1
Croissant & Brezel              1
Name: count, Length: 1339, dtype: int64

In [ ]:
item_similar_queries=[]
item_embeddings = items['embeddings_jointText']
for item in item_embeddings:
    top_indices, top_scores = semantic_search(item, query_gallery_embedding, top_k=3)
    query1=f" {queries['search_term_pt'][top_indices[0]]}, {queries['search_term_pt'][top_indices[2]]}, {queries['search_term_pt'][top_indices[2]]}"
    item_similar_queries.append(query1)